## KAGGLE BIKE SHARE CHALLENGE - Prediction Models - Machine Learning Models

For TpT Interview Assignment - June 2017

By: Divya Sriram

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

In [2]:
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

In [5]:
#libraries I may need for training + modeling
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.grid_search import GridSearchCV
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier

In [15]:
# Read in training and dev data sets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [16]:
print train.shape
print test.shape

(10886, 12)
(6493, 9)


In [17]:
#create new data frames and keep original train and test sets intact for reference

train_data = train
test_data = test

#separating dates into year, month, day, hour
train_data["datetime"] = pd.to_datetime(train_data["datetime"])
train_data["year"] = train_data["datetime"].dt.year
train_data["month"] = train_data["datetime"].dt.month
train_data["day"] = train_data["datetime"].dt.day
train_data["hour"] = train_data["datetime"].dt.hour

In [18]:
train_data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4


In [19]:
#splitting train_data into train and dev
# I believe the scikit learn's train_test_split shuffles the data in this step 
train_data, dev_data = train_test_split(train_data, test_size = 0.2)

In [20]:
#creating the labels
train_labels = train_data['count']
dev_labels = dev_data['count']

In [21]:
#dropping the prediction label (count) for the train data set
train_data = train_data.drop('count', 1)
dev_data = dev_data.drop('count', 1)

I will divide the train_data set into a training and development (dev) set: 80,20 respectively. I will use the new train_data set (80% of original train_data set) to train my model and use the dev set (20% of th e original train_data set) to test the model. This allows you to test your model against a set of data you have kept aside and make changes accordingly -- using the entire training data might result in a greater accuracy for that particular training set, but is highly prone to overfitting to the training data and performing poorly with test data. 

In [22]:
#prep the test set like we did train_data
test_data = test
test_data["datetime"] = pd.to_datetime(test_data["datetime"])
test_data["year"] = test_data["datetime"].dt.year
test_data["month"] = test_data["datetime"].dt.month
test_data["day"] = test_data["datetime"].dt.day
test_data["hour"] = test_data["datetime"].dt.hour

In [23]:
print "The size of the training data set is: ", train_data.shape
print "The size of the dev data set is: ", dev_data.shape
print "The size of the test data set is: ", test.shape

The size of the training data set is:  (8708, 15)
The size of the dev data set is:  (2178, 15)
The size of the test data set is:  (6493, 13)
